In [87]:
import pickle
import networkx as nx
import numpy as np
import pandas as pd
# import math
# import EoN
# import random
import pickle
from Phage_api import settings_local as local_settings
import glob

In [88]:
home = local_settings.CRUSTDB_DATABASE+'topology/'
home

'/home/platform/project/crustdb_platform/crustdb_api/workspace/crustdb_database/topology/'

In [89]:
# BFS
def my_hierarchy(node_map, G, root, parent = None):
    children = list(G.neighbors(root))
    if not isinstance(G, nx.DiGraph) and parent is not None:
        children.remove(parent)
    child_list = []
    if len(children)!=0:
        for child in children:
            x = my_hierarchy(node_map, G, child, parent=root)
            if x:
                child_list.append({
                    'name': child,
                    'value': node_map[child],
                    'children': x,
                })
            else:
                child_list.append({
                    'name': child,
                    'value': node_map[child],
                    # 'children': x,
                })
        return child_list
    else:
        return None


In [90]:
def generate_parent_child_relation(species, conformation):
    _nodes = pd.read_csv(home+species+'/'+'/'+conformation+'/MST/node.csv').to_numpy()
    node_map = {}
    for i in range(len(_nodes)):
        node_map[_nodes[i][0]] = _nodes[i][-1]

    path = home+species+'/'+'/'+conformation+'/MST/MST.pkl'
    with open(path, 'rb') as handle:
        G = pickle.load(handle)
    
    node_size = nx.pagerank(G, alpha=0.85,weight=None)
    sorted_dict = sorted(node_size.items(), key=lambda x: x[1],reverse=True)
    labels = {}
    for i in sorted_dict[:10]:
        labels[i[0]] = i[0]
    root = list(labels.values())[0]

    _res = my_hierarchy(node_map=node_map, G=G, root=root)
    res = {
        'name': root,
        'value': node_map[root],
        'children': _res,
    }

    return res

In [91]:
layer1 = glob.glob(home+'*')
for _species in layer1:
    species = _species.split('/')[-1]
    layer2 = glob.glob(_species+'/*')
    for _conformation in layer2:
        conformation = _conformation.split('/')[-1]
        try:
            res = generate_parent_child_relation(species=species, conformation=conformation)
            with open(home+species+'/'+conformation+'/MST/mst_parentchild_relation.pkl', 'wb') as handle:
                pickle.dump(res, handle, protocol=pickle.HIGHEST_PROTOCOL)
        except:
            print('Error ---> '+species+'/'+conformation)

Error ---> Lung/Lung12.gem.txt.cell_type.Treg_RRH9
